In [ ]:
# Import Libs
import seaborn as sns
import pandas as pd
import matplotlib.pyplot as plt
import graphviz
import copy
import numpy
import time
import altair as alt

from sklearn import datasets, tree
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
from sklearn.neighbors import KNeighborsClassifier
from os import system
from IPython.display import Image

%matplotlib inline

In [ ]:
# Import the data and name the collums
adultDf = pd.read_csv("adult.data",header=None, index_col=False, 
                      names=['age', 'workclass', 'fnlwgt', "education", "education_num", 
                             "martial-stat", "occupation", "relationship", "race", "sex", 
                             "cap-gains", "cap_loss", "hpw", "native_country", "50k"])

In [ ]:
# Give us a first look and check wether it looks ok
adultDf

In [ ]:
# Replace some Data to make it easier to work with:
# Add an Index:
# adultDf['Index'] = adultDf.index
# >50k -> 1, <=50k ->0
adultDf['50k'] = adultDf['50k'].replace(to_replace=' >50K', value=1)
adultDf['50k'] = adultDf['50k'].replace(to_replace=' <=50K', value=0)

# Education itself is roughly Ordinal, so lets Order it relativly arbitrarily and replace them with Numbers 
# according to the following, where we give the assoiate degrees amd Some collega the same value as they 
# are all rougly equal:
# Preschool, 1st-4th, 5th-6th, 7th-8th, 9th, 10th, 11th, 12th, HS-grad, Assoc-voc, Assoc-acdm, 
# Some-college, Bachelors, Masters, Prof-school, Doctorate,  
adultDf['education'] = adultDf['education'].replace(to_replace=' Preschool', value=0 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 1st-4th', value=1)
adultDf['education'] = adultDf['education'].replace(to_replace=' 5th-6th', value=2 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 7th-8th', value=3)
adultDf['education'] = adultDf['education'].replace(to_replace=' 9th', value=4 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 10th', value=5)
adultDf['education'] = adultDf['education'].replace(to_replace=' 11th', value=6 )
adultDf['education'] = adultDf['education'].replace(to_replace=' 12th', value=7 )
adultDf['education'] = adultDf['education'].replace(to_replace=' HS-grad', value=8 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Assoc-voc', value=9)
adultDf['education'] = adultDf['education'].replace(to_replace=' Assoc-acdm', value=9 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Some-college', value=9 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Bachelors', value=10)
adultDf['education'] = adultDf['education'].replace(to_replace=' Masters', value=11 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Prof-school', value=12 )
adultDf['education'] = adultDf['education'].replace(to_replace=' Doctorate', value=13 )
# Then print the New Dataframe to check:
adultDf

### Now lets Plot Stuff:


#### First Age Education and Income Correlation

In [ ]:
g = sns.FacetGrid(adultDf, hue='50k', height=5)
g.map(sns.scatterplot, "education", "age" )
g.add_legend()

#### We can see that apperantly Both Age and Education correlate with making over 50k, with Education being very Important 

In [ ]:
g = sns.FacetGrid(adultDf, height=5)
g.map(sns.scatterplot, "education", "education_num" )
g.add_legend()

##### Eductaion and Education_Num are basically the same. So we can ignore Education and solly focus on Education_num

In [ ]:
sns.catplot( y="fnlwgt", x='50k', kind="box", data=adultDf, height=10)

##### It seams whatever fnlwgt is its pretty much unkorrelated with our question. 

##### Lets Check for general Class imbalance and Sex Imbalance

In [ ]:
print("# of all people: " + str( len(adultDf) ) )
adultDf50k = adultDf[ adultDf['50k']==1 ]
print("# of People >50k: " + str( len(adultDf50k) ) )
print("# of Woman: " + str( len(adultDf[adultDf['sex'] == " Female" ]) ) )
print("# of Man: " + str( len(adultDf[adultDf['sex'] == " Male" ]) ) )
print("# of Woman >50k: " + str( len(adultDf50k[adultDf50k['sex'] == " Female" ]) ) )
print("# of Man >50k: " + str( len(adultDf50k[adultDf50k['sex'] == " Male" ]) ) )

### As we see there is a strong Class inbalance in the Data.

#### 1. only ~ 1/4 is over 50k, so a cassifier just saying "below 50k" will have a 75% accuracy
#### 2. Woman are generally underrepresented ~2:1
#### 3. In the group of people over 50k woman are underrepresented ~6:1, so another ~3:1 more than we expect for a 2:1 distribiution. 

#### This will propbably cause problems which we will have to adress in 3.5

In [ ]:
sns.catplot( y='sex', hue='50k', kind="count", height=6, aspect=2, data=adultDf)

In [ ]:
### Lets Check for an Sex, Race imbalance in eductation and Income:
g = sns.FacetGrid(adultDf, col="sex", hue='50k', height=5)
g.map(plt.scatter, "education_num", "age")

h = sns.FacetGrid(adultDf, col="race", hue='50k',height=5, col_wrap=2 )
h.map(plt.scatter, "education_num", "age" )

#### We see that Males are on Average more in the 50K group, than Females.
#### We see that Whites and Asians even more are better educated than Blacks Others, which as we already know correlates with higher Wages
#### We see that we have way more Data for White and Black than for the Others, with Asians in the Middle

In [ ]:
occupationstotal = adultDf['occupation'].value_counts()
occupation50k = adultDf.loc[adultDf['50k'] == 1]['occupation'].value_counts()
#print(occupationstotal)
#print(occupation50k)
occuppercentages = round(occupation50k / occupationstotal * 100,2)
print(occuppercentages)

#### We can see for Occupation that there are High paying fields with >35%, middle fields with 10%-35% and low paying fields <10%

In [ ]:
sns.catplot( y='workclass', hue='50k', kind="count", height=6, aspect=2, data=adultDf)
sns.catplot( y='occupation', hue='50k', kind="count", height=6, aspect=2, data=adultDf)
sns.catplot( y='martial-stat', hue='50k', kind="count", height=6, aspect=2, data=adultDf)
sns.catplot( y='relationship', hue='50k', kind="count", height=6, aspect=2, data=adultDf)

#### We see Workclass and Occupation obviously matter, with Occupation beeing very Important

#### We also see that relationship status (and marital status which overlap)  have significant influence on the income, although this might be an indirect correlation through Age

#### Lets Test this:

In [ ]:
sns.catplot( y="age", x='martial-stat', kind="box", data=adultDf, height=7, aspect= 2.5)

#### As we see a stable martial Relationship is Older than non Relationships, but Widdowed People are even Older and their high Income is rarer than of the married ones

#### Lets Take a Look at the Working Hours: 

In [ ]:
sns.catplot( y="hpw", x='50k', kind="box", data=adultDf, height=10, aspect= 1)

#### Those who make over 50k work on average and in the mean a couple hours more. But the Spread of both is quite high


#### Lets Check Capital Gains and Capital Losses:

In [ ]:
g = sns.FacetGrid(adultDf, hue='50k', height=5)
g.map(sns.scatterplot, "cap_loss", "cap-gains")
g.add_legend()

#### Ok, obviously people have either one or the other, but not both, we could therefor simplfy into: cap-gain-new = cap-gain - caploss
#### Also who is this person who made >40k in capital gains jet had <=50k income? Probably a Retiree who does not work anymore.

#### Finally lets Check the Native Country

In [ ]:
sns.catplot( y='native_country', hue='50k', kind="count", height=6, aspect=2, data=adultDf)

#### Great, basically all our Data is from the US, this is strong Class imbalance. Lets first check the Rest of People

In [ ]:
adultDf_nonUS = adultDf.copy()
adultDf_nonUS = adultDf_nonUS[adultDf_nonUS.native_country != ' United-States']
sns.catplot( y='native_country', hue='50k', kind="count", height=6, aspect=2, data=adultDf_nonUS)

### Here we can now see that some Countries like Japan have a high ratio of high earners, while other countries like Mexico have a high ratio nof low earners. So this is an important information

In [ ]:
matrix = adultDf.corr(method='kendall')
mask = numpy.triu(numpy.ones_like(matrix, dtype=bool))
cmap = sns.diverging_palette(230, 20, as_cmap=True)
sns.heatmap(matrix, mask=mask, cmap=cmap, vmax=.3, center=0, square=True, linewidths=.5, cbar_kws={"shrink": .5})

In [ ]:
adultDf_num = adultDf.copy()
cat_columns = adultDf_num .select_dtypes(['object']).columns
adultDf_num [cat_columns] = adultDf_num [cat_columns].apply(lambda x: pd.factorize(x)[0])

corrMatrix = adultDf_num .corr()

In [ ]:
# Generate a mask for the upper triangle
mask = numpy.triu(numpy.ones_like(corrMatrix, dtype=bool))

# Set up the matplotlib figure
f, ax = plt.subplots(figsize=(14, 14))

# Generate a custom diverging colormap
cmap = sns.diverging_palette(230, 20, as_cmap=True)

# Draw the heatmap with the mask and correct aspect ratio
sns.heatmap(corrMatrix, mask=mask, cmap=cmap, vmax=.45,vmin=-0.45, center=0,
            square=True, annot=corrMatrix, linewidths=.5, cbar_kws={"shrink": .3})

### Lets look now at some normalized graphs to ensure our findings are true:

#### We do this using altair, because of slight differences in internal functionality we reimport the data etc...

In [ ]:
colnames=['age','workclass','fnlwgt','education','education-num','marital-status','occupation','relationship','race','sex','capital-gain','capital-loss','hours-per-week','native-country', 'income']
df = pd.read_csv('adult.data', names=colnames, header=None)
df.head()

In [ ]:
df.info()
# Remove invalid data from table
rows_to_remove = len(df) - len(df[(df.astype(str) != ' ?').all(axis=1)])
print('Removing ',rows_to_remove,'rows, containing \"?\" in one of the columns.') 
df = df[(df.astype(str) != ' ?').all(axis=1)]
print(len(df), 'rows remaining')

#### Lets Look ate the Normaized Bargraphs

In [ ]:
# One of us needed one of those comands, the other one the other. 
# If this block does not work, comment one out and the other in.
#alt.data_transformers.enable('data_server')
alt.data_transformers.enable('default', max_rows=None)

In [ ]:
age_normd = alt.Chart(df).mark_bar().encode(
    x=alt.X('count():Q', stack="normalize"),
    y=alt.Y('age:Q', bin=True),
    color='income'
)

age = alt.hconcat(age_normd).properties(
 title="Age",
)
age

In [ ]:
workclass_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("workclass:N", sort=alt.EncodingSortField(field="workclass", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income'
)

workclassN = (workclass_normd).properties(
 title="Workclass Normed"
)
workclassN


In [ ]:
occupation_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("occupation:N", sort=alt.EncodingSortField(field="occupation", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income:N'
)

occupation = (occupation_normd).properties(
 title="Occupation",
)
occupation

In [ ]:
race_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("race:N", sort=alt.EncodingSortField(field="race", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income'
)

race = (race_normd).properties(
 title="Race",
)
race

In [ ]:
country_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("native-country:N", sort=alt.EncodingSortField(field="native-country", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income'
)

country = (country_normd).properties(
 title="Native Country",
)
country

In [ ]:
maritalstatus_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("marital-status:O", sort=alt.EncodingSortField(field="marital-status", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income'
)

maritalstatus = (maritalstatus_normd).properties(
 title="Marital Status",
)
maritalstatus

In [ ]:
relationship_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("relationship:O", sort=alt.EncodingSortField(field="relationship", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income'
)

relationship = (relationship_normd).properties(
 title="Relationship",
)
relationship

In [ ]:
edu_normd =  alt.Chart(df).mark_bar().encode(
    alt.Y("education:N", sort=alt.EncodingSortField(field="education", op="count", order='descending')),
    x= alt.X('count()', stack="normalize"),
    color='income'
)

edu = (edu_normd).properties(
 title="Education",
)
edu

### Good News, the normalized graphs confirm the conclusions we came to earlier
